In [3]:
'''Create an XML tree for Oxford Pet set style annotation of files'''
import xml.etree.cElementTree as ET

annotation1 = ET.Element("annotation")

folder = ET.SubElement(annotation1, "folder").text = "dummy text"
filename = ET.SubElement(annotation1, "filename").text = "dummy text"
path = ET.SubElement(annotation1, "path").text = "dummy text"
source = ET.SubElement(annotation1, "source")

database = ET.SubElement(source, "database").text = "dummy text"
annotation2 = ET.SubElement(source, "annotation").text = "dummy text"
image = ET.SubElement(source, "image").text = "dummy text"

size = ET.SubElement(annotation1, "size")

width = ET.SubElement(size, "width").text = "dummy text"
height = ET.SubElement(size, "height").text = "dummy text"
depth = ET.SubElement(size, "depth").text = "dummy text"

segmented = ET.SubElement(annotation1, "segmented").text = "dummy text"
object1 = ET.SubElement(annotation1, "object")

name = ET.SubElement(object1, "name").text = "dummy text"
pose = ET.SubElement(object1, "pose").text = "dummy text"
truncated = ET.SubElement(object1, "truncated").text = "dummy text"
occluded = ET.SubElement(object1, "occluded").text = "dummy text"
bndbox = ET.SubElement(object1, "bndbox")

xmin = ET.SubElement(bndbox, "xmin").text = "dummy text"
ymin = ET.SubElement(bndbox, "ymin").text = "dummy text"
xmax = ET.SubElement(bndbox, "xmax").text = "dummy text"
ymax = ET.SubElement(bndbox, "ymax").text = "dummy text"

difficult = ET.SubElement(object1, "difficult").text = "dummy text"

tree = ET.ElementTree(annotation1)
tree.write("test1.xml")

In [36]:
import xml.etree.cElementTree as ET
import pandas as pd
import os
from PIL import Image

In [34]:
for subdir, dirs, files in os.walk('raw_data/train'):
        for file in files:
            ext = os.path.splitext(file)[-1].lower()
            if ext == ".csv":
                img_folder = os.path.join(subdir,file)[:-4]
#                 for subdir, dirs, files in os.walk(img_folder):
#                     print(files)
#                 print(subdir)
                df = pd.read_csv(os.path.join(subdir,file))
                ind = df[df.ObjectType.isnull()].index
                df = df.query('index not in @ind')
                for frame in df.Frame.unique(): # Perform operation for each frame
                    print(os.path.abspath(os.path.join(img_folder,'{:06}.png'.format(frame))))

/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/train/Neovision2-Training-Heli-001/000000.png
/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/train/Neovision2-Training-Heli-001/000004.png
/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/train/Neovision2-Training-Heli-001/000008.png
/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/train/Neovision2-Training-Heli-001/000012.png
/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/train/Neovision2-Training-Heli-001/000016.png
/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/train/Neovision2-Training-Heli-001/000020.png
/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/train/Neovision2-Training-Heli-001/000024.png
/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/train/Neovision2-Training-Heli-001/000028.png
/Users/aleksod/Documents/Metis/M

In [17]:
# ext1 = ['.csv']
for subdir, dirs, files in os.walk('raw_data/train'):
        for file in files:
            ext = os.path.splitext(file)[-1].lower()
            if ext == ".csv":
                img_folder = print(os.path.join(subdir,file)[:-4])
#                 print(os.path.join(subdir,file))
                df = pd.read_csv(os.path.join(subdir,file))
                ind = df[df.ObjectType.isnull()].index
                df = df.query('index not in @ind')
                for frame in df.Frame.unique(): # Perform operation for each frame
                    
                    image_filename = os.path.abspath(os.path.join(img_folder,'{:06}.png'.format(frame)))
                    im = Image.open(image_filename)
                    width, height = im.size
                    
                    annotation1 = ET.Element("annotation")

                    folder = ET.SubElement(annotation1, "folder").text = "images"
                    filename = ET.SubElement(annotation1, "filename").text = '{:06}.png'.format(frame)
                    path = ET.SubElement(annotation1, "path").text = image_filename
                    source = ET.SubElement(annotation1, "source")

                    database = ET.SubElement(source, "database").text = "Unknown"
                    annotation2 = ET.SubElement(source, "annotation").text = "dummy text"
                    image = ET.SubElement(source, "image").text = "dummy text"

                    size = ET.SubElement(annotation1, "size")

                    width = ET.SubElement(size, "width").text = "dummy text"
                    height = ET.SubElement(size, "height").text = "dummy text"
                    depth = ET.SubElement(size, "depth").text = "dummy text"

                    segmented = ET.SubElement(annotation1, "segmented").text = "dummy text"
                    object1 = ET.SubElement(annotation1, "object")

                    name = ET.SubElement(object1, "name").text = "dummy text"
                    pose = ET.SubElement(object1, "pose").text = "dummy text"
                    truncated = ET.SubElement(object1, "truncated").text = "dummy text"
                    occluded = ET.SubElement(object1, "occluded").text = "dummy text"
                    bndbox = ET.SubElement(object1, "bndbox")

                    xmin = ET.SubElement(bndbox, "xmin").text = "dummy text"
                    ymin = ET.SubElement(bndbox, "ymin").text = "dummy text"
                    xmax = ET.SubElement(bndbox, "xmax").text = "dummy text"
                    ymax = ET.SubElement(bndbox, "ymax").text = "dummy text"

                    difficult = ET.SubElement(object1, "difficult").text = "dummy text"
                    
                    print('frame:', frame)
                    df2 = df[df.Frame == frame]
                    filename = "temp/frame%d.jpg" % frame
                    image_format = b'jpg'

                    width = max(image.shape)
                    height = min(image.shape)
                    print(width, height)

                    df3x = df2.iloc[:,[1,3,5,7]]
                    df3y = df2.iloc[:,[2,4,6,8]]

                    xmins_abs = list(df3x.apply(lambda x: min(x), axis=1))
                    ymins_abs = list(df3y.apply(lambda y: min(y), axis=1))
                    xmaxs_abs = list(df3x.apply(lambda x: max(x), axis=1))
                    ymaxs_abs = list(df3y.apply(lambda y: max(y), axis=1))

                    xmins = [x/width  for x in xmins_abs]
                    ymins = [y/height for y in ymins_abs]
                    xmaxs = [x/width  for x in xmaxs_abs]
                    ymaxs = [y/height for y in ymaxs_abs]

                    classes_text = list(df2.iloc[:,9])
                    print(classes_text)
                    classes  = [classes_labels[x] for x in classes_text]
                    classes_text

                    del df2, df3x, df3y
                    gc.collect()
                
                tree = ET.ElementTree(annotation1)
                tree.write(os.path.join(os.path.abspath(os.path.join(img_folder,'{:06}.xml'.format(frame)))))

raw_data/train/Neovision2-Training-Heli-001.csv
raw_data/train/Neovision2-Training-Heli-007.csv


In [24]:
os.path.join(subdir,file)

'raw_data/train/Neovision2-Training-Heli-007/000449.png'

In [23]:
df.Frame.unique()[1]

4

In [18]:
df.head()

,Frame,BoundingBox_X1,BoundingBox_Y1,BoundingBox_X2,BoundingBox_Y2,BoundingBox_X3,BoundingBox_Y3,BoundingBox_X4,BoundingBox_Y4,ObjectType,Occlusion,Ambiguous,Confidence,SiteInfo,Version
0,0,277,360,417,369,413,419,274,410,Car,False,False,1.0,NaN,1.4
1,0,468,361,624,377,617,442,461,426,Car,False,False,1.0,NaN,1.4
2,4,99,371,239,380,235,430,96,421,Car,False,False,1.0,NaN,1.4
3,4,294,378,450,394,443,459,287,443,Car,False,False,1.0,NaN,1.4
4,4,1722,730,1807,730,1807,786,1722,786,Person,False,False,1.0,NaN,1.4


In [ ]:
df = pd.read_csv(video_name.replace(extensions[0], '.csv'))
ind = df[df.ObjectType.isnull()].index
df = df.query('index not in @ind')

'''Walk down each frame mentioned in the CSV and mark rectangles with objects for model training'''

for index,frame in enumerate(df.Frame.unique()): # Perform operation for each frame
    print('frame:', frame)
    df2 = df[df.Frame == frame]
    filename = "temp/frame%d.jpg" % frame
    image_format = b'jpg'

    width = max(image.shape)
    height = min(image.shape)
    print(width, height)

    df3x = df2.iloc[:,[1,3,5,7]]
    df3y = df2.iloc[:,[2,4,6,8]]

    xmins_abs = list(df3x.apply(lambda x: min(x), axis=1))
    ymins_abs = list(df3y.apply(lambda y: min(y), axis=1))
    xmaxs_abs = list(df3x.apply(lambda x: max(x), axis=1))
    ymaxs_abs = list(df3y.apply(lambda y: max(y), axis=1))

    xmins = [x/width  for x in xmins_abs]
    ymins = [y/height for y in ymins_abs]
    xmaxs = [x/width  for x in xmaxs_abs]
    ymaxs = [y/height for y in ymaxs_abs]

    classes_text = list(df2.iloc[:,9])
    print(classes_text)
    classes  = [classes_labels[x] for x in classes_text]
    classes_text

    del df2, df3x, df3y
    gc.collect()

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), 'annotations')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('raccoon_labels.csv', index=None)
    print('Successfully converted xml to csv.')


main()